### Import required libraries

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1, 3"

import numpy as np
import scipy as sp
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

import keras
import tensorflow as tf

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_curve, auc

from keras import optimizers
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.layers import Conv2D, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.utils import multi_gpu_model

from densenet import *

/home/jackluo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/jackluo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# % matplotlib inline
# % config InlineBackend.figure_format = 'retina'

### Configuration settings

In [24]:
SEED = 1

COLOR = 'rgb'
IMG_SIZE = (299, 299)

TRAIN_BATCH_SIZE = 48
VALID_BATCH_SIZE = 48
TEST_BATCH_SIZE = 128

NB_EPOCHS = 100

LR = 0.03
NB_SNAPSHOTS = 5

REDUCTION_FACTOR = 0.5
REDUCTION_PATIENCE = 1
MINIMUM_LR = 1e-6

# WEIGHTS = None
WEIGHTS = 'imagenet'

In [4]:
core_idg = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range=0.1, 
                              width_shift_range=0.05, 
                              shear_range=5,
                              zoom_range=[0.94, 0.98],
                              rotation_range=5,
                              fill_mode = 'constant'
                             )

val_idg = ImageDataGenerator(samplewise_center=True, 
                             samplewise_std_normalization=True, 
                             zoom_range=[0.96, 0.96]
                            )

In [5]:
all_labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 
              'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

# all_labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']

data_dir = 'data'

_dir = data_dir + '/split/' + 'df.csv'
df = pd.read_csv(_dir)

### Important functions

In [6]:
def get_sample_counts(df, class_names):

    total_count = df.shape[0]
    labels = df[class_names].as_matrix()
    positive_counts = np.sum(labels, axis=0)
    class_positive_counts = dict(zip(class_names, positive_counts))
    
    return total_count, class_positive_counts

In [7]:
def get_class_weights(total_counts, class_positive_counts, multiply):

    def get_single_class_weight(pos_counts, total_counts):
        denominator = (total_counts - pos_counts) * multiply + pos_counts
        return {
            0: pos_counts / denominator,
            1: (denominator - pos_counts) / denominator,
        }

    class_names = list(class_positive_counts.keys())
    label_counts = np.array(list(class_positive_counts.values()))
    class_weights = []
    for i, class_name in enumerate(class_names):
        class_weights.append(get_single_class_weight(label_counts[i], total_counts))

    return class_weights

In [8]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, b, s):
    
    base_dir = os.path.dirname(in_df[path_col].values[0])
 
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                              class_mode='sparse',
                                              target_size=IMG_SIZE,
                                              color_mode=COLOR,
                                              batch_size=b,
                                              shuffle=s)
    
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    
    return df_gen

In [9]:
def cosine_anneal_schedule(epoch):
    
    cos_inner = np.pi * (epoch % (NB_EPOCHS // NB_SNAPSHOTS))
    cos_inner /= NB_EPOCHS // NB_SNAPSHOTS
    cos_outer = np.cos(cos_inner) + 1
    
    return float(LR / 2 * cos_outer)

### Load data

In [10]:
df['disease_vec'] = df.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])

In [11]:
# df_train, second_df = train_test_split(df, 
#                                        test_size=0.3, 
#                                        random_state=SEED)

# df_valid, df_test = train_test_split(second_df, 
#                                      test_size=0.6666666666, 
#                                      random_state=SEED)

np.random.seed(SEED)

unique_users = df['Patient ID'].unique()
train_users, valid_users, test_users = np.split(
    np.random.permutation(unique_users), [int(.7 * len(unique_users)), int(.8 * len(unique_users))]
)

df_train = df[df['Patient ID'].isin(train_users)]
df_valid = df[df['Patient ID'].isin(valid_users)]
df_test = df[df['Patient ID'].isin(test_users)]

print('Train: ', df_train.shape[0])
print('Validation: ', df_valid.shape[0])
print('Test:', df_test.shape[0])

Train:  78427
Validation:  11747
Test: 21946


In [12]:
train_gen = flow_from_dataframe(core_idg, 
                                df_train, 
                                path_col='path',
                                y_col='disease_vec',
                                b=TRAIN_BATCH_SIZE, 
                                s=True)

valid_gen = flow_from_dataframe(val_idg, 
                                df_valid, 
                                path_col='path',
                                y_col='disease_vec',
                                b=VALID_BATCH_SIZE, 
                                s=False)

test_gen = flow_from_dataframe(val_idg, 
                               df_test, 
                               path_col='path',
                               y_col='disease_vec',
                               b=TEST_BATCH_SIZE, 
                               s=False)

Found 0 images belonging to 0 classes.
Reinserting dataframe: 78427 images
Found 0 images belonging to 0 classes.
Reinserting dataframe: 11747 images
Found 0 images belonging to 0 classes.
Reinserting dataframe: 21946 images


In [13]:
X_train, y_train = next(train_gen)
X_valid, y_valid = next(valid_gen)

# fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
# for (c_x, c_y, c_ax) in zip(X_train, y_train, m_axs.flatten()):
#      c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = -1.5, vmax = 1.5)
#      c_ax.set_title(', '.join([n_class for n_class, n_score in zip(all_labels, c_y) 
#                                if n_score>0.5]))
#      c_ax.axis('off')

In [14]:
train_counts, train_pos_counts = get_sample_counts(df_train, all_labels)
class_weights = get_class_weights(train_counts, train_pos_counts, multiply=1)

class_weights

[{0: 0.1015594119372155, 1: 0.8984405880627845},
 {0: 0.023601565787292642, 1: 0.9763984342127073},
 {0: 0.04210284723373328, 1: 0.9578971527662667},
 {0: 0.01981460466421003, 1: 0.98018539533579},
 {0: 0.11938490570849325, 1: 0.8806150942915068},
 {0: 0.02140844352072628, 1: 0.9785915564792738},
 {0: 0.015211598046591097, 1: 0.9847884019534089},
 {0: 0.002078365868897191, 1: 0.9979216341311028},
 {0: 0.17667384956711338, 1: 0.8233261504328866},
 {0: 0.05207390312009894, 1: 0.947926096879901},
 {0: 0.0571104339066903, 1: 0.9428895660933097},
 {0: 0.030882221683858874, 1: 0.9691177783161411},
 {0: 0.012495696635087407, 1: 0.9875043033649126},
 {0: 0.047088375176916115, 1: 0.9529116248230839}]

### Build model

In [15]:
adam = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

sgd = optimizers.SGD(decay=1e-5, momentum=0.9, nesterov=True)

In [16]:
# base_model = DenseNet([6, 12, 48, 32], growth_rate=32,
#              include_top=False, weights=WEIGHTS, 
#              input_tensor=None, input_shape=X_train.shape[1:], 
#              pooling=None, classes=len(all_labels))

base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False, weights=WEIGHTS, 
    input_tensor=None, input_shape=X_train.shape[1:], 
    pooling=None, classes=len(all_labels)
)

# x = GlobalMaxPooling2D()(base_model.output)
x = GlobalAveragePooling2D()(base_model.output)
predictions = Dense(len(all_labels), activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
parallel_model = multi_gpu_model(model, gpus=3)

# parallel_model.load_weights('weights.h5')
parallel_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy', 'categorical_accuracy'])

In [20]:
checkpoint = ModelCheckpoint('weights_{epoch:02d}.h5', monitor='val_loss', verbose=1, 
                             save_best_only=False, mode='min', save_weights_only=True)

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=REDUCTION_FACTOR, patience=REDUCTION_PATIENCE,
                              verbose=1, mode="min", min_lr=MINIMUM_LR)

lr_schedule = LearningRateScheduler(schedule=cosine_anneal_schedule, verbose=1)

callbacks_list = [checkpoint, lr_schedule]

In [21]:
# model.summary()

In [ ]:
parallel_model.fit_generator(train_gen, 
                             initial_epoch=0,
                             steps_per_epoch=int(df_train.shape[0]/TRAIN_BATCH_SIZE),
                             epochs=NB_EPOCHS,  
                             validation_data=valid_gen, 
                             validation_steps=int(df_valid.shape[0]/VALID_BATCH_SIZE),
                             class_weight=class_weights,
                             callbacks=callbacks_list)

Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.03.
 645/1633 [==========>...................] - ETA: 25:02 - loss: 0.1734 - acc: 0.9466 - categorical_accuracy: 0.1813

In [ ]:
# parallel_model.load_weights('weights.h5')

model.save('model.h5')

### Evaluate model

In [ ]:
y_test_df = df_test['disease_vec'].values

y_test = []

for i in y_test_df:
    y_test.append(i)
    
y_test = np.array(y_test)
y_test.shape

In [ ]:
y_pred = parallel_model.predict_generator(test_gen, verbose=True)

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'

fig, ax = plt.subplots(1,1, figsize=(9, 9))

for (idx, c_label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(y_test[:, idx].astype(int), y_pred[:, idx])
    ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    
ax.plot([0, 1], [0, 1], color='grey', lw=2, linestyle='--')
ax.set_title('ROC Curve')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.set_xlim([-0.01, 1.01])
ax.set_ylim([-0.01, 1.01])

fig.savefig('roc.png')